# Updating stock in Woocommerce from Edisoft with Python

This Python script is designed to update the stock in a web store using the WooCommerce REST API. The script takes stock data from a program called Edisoft and compares it with the stock data in the web store. If there are any differences, the script updates the stock in the web store.

Let's go through the script step by step:

1. The necessary libraries are imported, including pandas, numpy, re, woocommerce, smtplib, and others.

2. Parameter values are set, including the location of the stock files, email parameters, and WooCommerce parameters.

3. The script defines a function called **`enviar_email`** which is responsible for sending an email using SMTP with the given information.

4. A decorator function called **`log_exception`** is defined. This decorator logs any exceptions raised by a given function and sends an email with the error message.

5. The **`file_clean`** function is defined to eliminate rows that are not inventory lines in a file. It uses a regular expression pattern to filter out non-inventory lines.

6. The **`select_inventory`** function is defined to select the most recent file starting with "I20" within a specified folder. It compares the most recent file with the second most recent file and continues if there is a difference.

7. The **`import_clean_inventory`** function is defined to import and clean an Edisoft inventory dataset. It reads the dataset file, extracts information from columns based on fixed positions, converts stock and price columns to appropriate data types, removes duplicates based on ISBN, and returns the cleaned dataset as a pandas DataFrame.

8. The **`import_inventary_web`** function is defined to import the inventory from a WooCommerce webshop using the WooCommerce REST API. It creates a WooCommerce API object, downloads products from the webshop, transforms them into a DataFrame, removes duplicates based on product ID, and returns the imported inventory as a DataFrame.

9. The **`web_edisoft_join`** function is defined to select items with different stock between the web store inventory and the Edisoft inventory. It joins the two DataFrames based on SKU and ISBN columns, identifies the items with different stock, and returns a filtered DataFrame.

10. The **`stock_update_batch`** function is defined to update the stock of the web store using the WooCommerce REST API and verify the update. It creates a WooCommerce API object, divides the inventory DataFrame into batches to avoid exceeding the connection time, updates the stock using the API, and checks if the updated items have been properly updated.

11. The script calls the functions in the following sequence:
   a. **`import_clean_inventory`** function to import and clean the Edisoft inventory dataset.
   b. **`import_inventary_web`** function to import the inventory from the web store.
   c. **`web_edisoft_join`** function to select items with different stock between the web store and Edisoft inventories.
   d. **`stock_update_batch`** function to update the stock in the web store.

Overall, this script automates the process of updating the stock in a web store by comparing it with the stock data from Edisoft and using the WooCommerce REST API to make the necessary updates. It also handles exceptions and sends email notifications in case of errors.

## Import library

In [35]:
!pip install WooCommerce

In [36]:
import os
import pandas as pd
import numpy as np
import re
import math
from woocommerce import API
import filecmp
from datetime import date
import csv
import smtplib
from email.message import EmailMessage
from functools import wraps
from random import sample

## Parameter values

In [37]:
# location where the stock files are located
path = "W:/soft/exporsinli/"
# email parameters
sender = "yourmaailsender@xxx.com"
recipient = "yourmaailrecipient@xxx.com"
mail_pass = "yourpassword"
smtp_server = "yoursmtpserver"
bcc = "youremailbcc.xxx.com"
# woocommerce parameters
url = "https://yourwebsiteurl.com"
consumer_key = "ck_*****"
consumer_secret = "cs_*****"
timeout = 500

## Functions

### enviar_email()

This function **`enviar_email()`** sends an email using the Simple Mail Transfer Protocol (SMTP). It takes the following arguments:

* **`info`** (str): Information about the email.
* **`sender`** (str): Sender's email address.
* **`recipient`** (str): Recipient's email address.
* **`smtp_server`** (str): SMTP server address.
* **`mail_pass`** (str): Sender's email password.
* **`bcc`** (str, optional): Email address for BCC (blind carbon copy) recipient. Defaults to None.

The function constructs an email message using the EmailMessage class from the email.message module. It sets the From, To, Subject, and Content of the email based on the provided information.

The subject line of the email is generated dynamically by including the current date using the date.today() function from the datetime module.

The body of the email includes a formatted message with the current date, indicating the completion of a program for updating stock on a website, and the provided info string.

The function attempts to send the email by establishing a secure connection to the SMTP server using the smtplib.SMTP_SSL class. It logs in with the sender's email address and password and sends the constructed message using the send_message method of the mail_server object.

If there is an exception during the email sending process, it catches the smtplib.SMTPException, prints an error message, and returns False.

The function returns True if the email is sent successfully.

This function provides a convenient way for data scientists to send emails using SMTP with customizable sender, recipient, server, and password. It simplifies the process of constructing and sending email messages, making it easier to incorporate email notifications into data science workflows.

The function **`enviar_email()`** sends an email using the Simple Mail Transfer Protocol (SMTP). It takes the following arguments:

- **`info`** (str): Information about the email.
- **`sender`** (str): Sender's email address.
- **`recipient`** (str): Recipient's email address.
- **`smtp_server`** (str): SMTP server address.
- **`mail_pass`** (str): Sender's email password.
- **`bcc`** (str, optional): Email address for BCC (blind carbon copy) recipient. Defaults to None.

The function constructs an email message using the **`EmailMessage`** class from the **`email.message`** module. It sets the **`From`**, **`To`**, **`Subject`**, and **`Content`** of the email based on the provided information.

The subject line of the email is generated dynamically by including the current date using the **`date.today()`** function from the **`datetime`** module.

The body of the email includes a formatted message with the current date, indicating the completion of a program for updating stock on a website, and the provided **`info`** string.

The function attempts to send the email by establishing a secure connection to the SMTP server using the **`smtplib.SMTP_SSL`** class. It logs in with the sender's email address and password and sends the constructed message using the **`send_message`** method of the **`mail_server`** object.

If there is an exception during the email sending process, it catches the **`smtplib.SMTPException`**, prints an error message, and returns **`False`**.

The function returns **`True`** if the email is sent successfully.

In [38]:
def enviar_email(info,
                 sender=sender,
                 recipient=recipient,
                 smtp_server=smtp_server,
                 mail_pass=mail_pass,
                 bcc=None):
    """
    Sends an email using SMTP with the given information.

    Args:
        info (str): Information about the email.
        sender (str): Sender's email address.
        recipient (str): Recipient's email address.
        smtp_server (str): SMTP server address.
        mail_pass (str): Sender's email password.
        bcc (str, optional): Email address for BCC recipient. Defaults to None.

    Returns:
        bool: True if the email is sent successfully, False otherwise.
    """
    message = EmailMessage()
    message["From"] = sender
    message["To"] = recipient
    if bcc:
        message["Bcc"] = bcc
    message["Subject"] = f"stock update {date.today()}"
    body = f"""The execution of the program to update the stock on the website
    on {date.today()} has finished.

    The result has been:

    {info}
    """
    message.set_content(body)

    try:
        with smtplib.SMTP_SSL(smtp_server) as mail_server:
            mail_server.set_debuglevel(1)
            mail_server.login(sender, mail_pass)
            mail_server.send_message(message)
    except smtplib.SMTPException as e:
        print("An error occurred while sending the email:", e)
        return False

    return True


### log_exception()

The **`log_exception`** function is a decorator that can be applied to other functions in order to log any exceptions that occur during their execution. It performs the following steps:

1. Decorator Setup: The **`@wraps(func)`** decorator ensures that the wrapper function maintains the same name and documentation as the original function being decorated.

2. Wrapper Function: The **`wrapper`** function is defined within the**`log_exception`** function. This function serves as a wrapper around the decorated function.

3. Exception Handling: Inside the **`wrapper`** function, a **`try-except`** block is used to catch any exceptions that may occur during the execution of the decorated function (**`func`**). If an exception is raised, it will be caught in the **`except`** block.

4. Logging: If an exception is caught, the function proceeds to log the error message in a CSV file. The **`log_file`** variable stores the name of the CSV file ("log_update_stock.csv"). The **`error_message`** variable is created, which includes the details of the error message and the name of the function where the exception occurred. The error message is appended to the CSV file using the **`csv.writer`** object.

5. Email Notification: After logging the exception, the **`enviar_email`** function is called to send an email with the error message. The **`error_message`** is passed as an argument to the **`enviar_email`** function.

6. Return: The **`wrapper`** function returns the result of the decorated function (**`func(*args, **kwargs)`**). If no exception occurs, the function execution continues as normal.

By using the **`log_exception`** decorator on a function, any exceptions that occur during the execution of that function will be caught, logged to a CSV file, and an email notification will be sent. This helps in tracking and analyzing the errors that occur during the execution of the decorated function.

In [39]:
def log_exception(func):
    """
    A decorator function that logs exceptions raised by a given function.

    Args:
        func (callable): The function to be decorated.

    Returns:
        callable: The decorated function.

    Raises:
        None.

    Examples:
        >>> @log_exception
        ... def my_function():
        ...     # code goes here
        ...
        >>> my_function()
        # If an exception is raised, it will be logged and an email will be sent.

    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            log_file = "log_update_stock.csv"
            error_message = f"The following error occurred: {e} in the function\
                             {func.__name__}\n"
            with open(log_file, "a", encoding="utf-8", newline='') as csv_file:
                csv_writer = csv.writer(csv_file)
                csv_writer.writerow([date.today(), error_message.strip()])
            enviar_email(error_message)
            exit()
    return wrapper

### file_clean()

The **`file_clean`** function takes a file path as input and modifies the file by removing rows that are not considered inventory lines.

Inside the function, a regular expression pattern **`^D[0-9]`** is defined, which matches lines starting with 'D' followed by a digit. This pattern is used to identify inventory lines.

The file is opened in read mode using **`open(file_path, 'r')`**, and the lines are read using the **`readlines()`** method. Then, the file is opened in write mode using **`open(file_path, 'w')`** to overwrite its contents.

A list comprehension is used to filter out non-inventory lines from the **`lines`** list. Only the lines that match the pattern are included in the **`inventory_lines`** list.

Finally, the **`writelines()`** method is used to write the **`inventory_lines`** back to the file, effectively removing the non-inventory lines.

The function does not return any value (**`None`**) but modifies the file directly.

In [40]:
@log_exception
def file_clean(file_path):
    """
    Void function for eliminating rows that are not inventory lines.

    Args:
        file_path (str): The path of the file.

    Returns:
        None

    Raises:
        None

    Examples:
        >>> file_clean('inventory.txt')
        # The function will modify the 'inventory.txt' file,
        # removing non-inventory lines.

    """
    pattern = '^D[0-9]'
    with open(file_path, 'r') as file:
        lines = file.readlines()
    with open(file_path, 'w') as file:
        inventory_lines = [line for line in lines if re.match(pattern, line)]
        file.writelines(inventory_lines)


### select_inventory()

The**`select_inventory`** function takes a**`path`** argument as input, representing the folder to search for files. It selects the most recent file starting with "I20" within the folder, compares it with the second most recent file, and returns the filename of the most recent file if there is a difference.

The function starts by initializing an empty**`file_list`** to store the filenames that start with "I20". It iterates over the files in the specified**`path`** using**`os.listdir(path)`** and checks if each filename starts with "I20". If a match is found, the full file path is created using**`os.path.join(path, filename)`** and added to the**`file_list`.

The**`file_list`** is then sorted based on the creation time of the files, with the most recent file appearing first. The filenames are sorted in descending order using the**`reverse=True`** parameter.

The most recent and second most recent filenames are extracted from the sorted**`file_list`. The function then proceeds to print the names of the files.

The**`file_clean`** function is called to clean both the most recent and second most recent files by eliminating rows that are not inventory lines.

Next, the function compares the content of the two files using**`filecmp.cmp(edisoft_today, edisoft_previous)`. If the files are identical, it prints a message, logs the update in the "log_update_stock.csv" file, sends an email, and exits the script.

Finally, if there are differences between the files or if the comparison step was skipped, the filename of the most recent file (`edisoft_today`) is returned.

Note: Ensure that you have implemented the**`file_clean`** function and have the necessary code to send emails (`env

iar_email`) and handle the logging of updates in the "log_update_stock.csv" file.

In [41]:
@log_exception
def select_inventory(path):
    """
    Selects the most recent file starting with "I20" within the specified folder.
    If there is a difference with the second most recent file, the function
    continues.

    Args:
        path (str): The path of the folder to search for files.

    Returns:
        str: The filename of the most recent file.

    Raises:
        None

    Examples:
        >>> select_inventory('/path/to/folder')
        # The function will locate the most recent "I20" file within the folder,
        # compare it with the second most recent file,
        # and return the filename of the most recent file if there is
        # a difference.

    """
    file_list = []
    for filename in os.listdir(path):
        if filename.startswith("I20"):
            path_file = os.path.join(path, filename)
            file_list.append(path_file)

    file_list = sorted(file_list, key=os.path.getctime, reverse=True)
    edisoft_today = file_list[0]
    edisoft_previous = file_list[1]

    print(f"Today's file name is {edisoft_today} and yesterday's file name\
            is {edisoft_previous}.")

    # Clean files
    file_clean(edisoft_today)
    file_clean(edisoft_previous)

    # Check for changes between the last two edisoft stock files
    if filecmp.cmp(edisoft_today, edisoft_previous):
        print("No update due to no changes - select_inventory")
        with open("log_update_stock.csv", "a", encoding="utf-8",
                  newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow([date.today(), "No update due to no changes -\
                                                select_inventory"])
        enviar_email("No update due to no changes - select_inventory")
        exit()

    return edisoft_today


### import_clean_inventory()


The **`import_clean_inventory`** function takes a **`path`** argument representing the path of the dataset file. It imports and cleans the edisoft inventory dataset with fixed position columns.

The function begins by calling the **`select_inventory`** function to get the most recent file in the specified path.

Then, the edisoft inventory dataset is read using **`pd.read_csv(edisoft_today, encoding="latin-1")`. The dataset is printed and displayed.

The script proceeds to extract information in columns based on fixed positions. Columns **`isbn`**, **`stock`**, and **`price`** are created using string slicing on the first column of the dataset.

Next, the script removes dashes from the **`isbn`** column and drops the original column using **`str.replace("-", "")`** and **`drop()`** methods, respectively.

The **`stock`** column is converted to integer type using **`.astype(int)`**, and the **`price`** column is converted to float type by dividing it by 1000 after converting it to integer type.

The cleaned dataset is printed and displayed.

Duplicates in the **`isbn`** column are checked by finding duplicated values. If duplicates exist,

 they are printed and displayed. The duplicates are then removed from the dataset using **`drop_duplicates()`**.

After cleaning, the function prints the dataset's information using **`info()`** method.

Finally, the cleaned dataset is returned as a pandas DataFrame.

In [42]:
@log_exception
def import_clean_inventory(path):
    """
    Imports and cleans an edisoft inventory dataset with fixed position columns.

    Args:
        path (str): The path of the dataset file.

    Returns:
        pd.DataFrame: The cleaned edisoft inventory dataset.

    Raises:
        None

    Examples:
        >>> import_clean_inventory('/path/to/dataset.csv')
        # The function will import the dataset, clean it by extracting columns with fixed positions,
        # convert stock and price columns to appropriate data types, remove duplicates based on ISBN,
        # and return the cleaned dataset as a DataFrame.

    """
    edisoft_today = select_inventory(path)

    # Import the edisoft inventory dataset
    edisoft_df = pd.read_csv(edisoft_today, encoding="latin-1")
    print("Let's take a look:\n", edisoft_df)
    print("-" * 100)

    # Extract information in columns based on fixed positions
    edisoft_df['isbn'] = edisoft_df.iloc[:, 0].str[1:18]
    edisoft_df['stock'] = edisoft_df.iloc[:, 0].str[18:25]
    edisoft_df['price'] = edisoft_df.iloc[:, 0].str[-10:-1]

    # Remove dashes and delete the original column
    edisoft_df['isbn'] = edisoft_df["isbn"].str.replace("-", "")
    edisoft_df.drop(edisoft_df.columns[0], axis=1, inplace=True)

    # Convert stock from text to int and price from text to float
    edisoft_df["stock"] = edisoft_df["stock"].astype(int)
    edisoft_df["price"] = edisoft_df["price"].astype(int) / 1000

    print("Let's take another look:\n", edisoft_df)
    print("-" * 100)

    # Check for duplicates based on ISBN
    duplicates = edisoft_df[edisoft_df["isbn"].duplicated()]
    if not duplicates.empty:
        print("Tenemos duplicados:\n", duplicates.sort_values("isbn"))
        edisoft_df.drop_duplicates(subset="isbn", inplace=True)
        duplicates_after_cleaning = edisoft_df[edisoft_df["isbn"].duplicated()]
        print("We have duplicates after cleaning:\n", duplicates_after_cleaning.sort_values("isbn"))

    print("Let's look at the type of data we have:\n", edisoft_df.info())
    return edisoft_df


### import_inventory_web()

This script is used to import the inventory from a WooCommerce webshop using the WooCommerce REST API. Let's go through the script step by step:

1. The function **`import_inventory_web`** is defined with four parameters: **`url`, **`consumer_key`**, **`consumer_secret`**, and **`timeout`**. These parameters are used to establish a connection with the WooCommerce webshop and retrieve the inventory data.

2. The function has a docstring that provides information about the purpose of the function, its arguments, return value, and examples of how to use it.

3. The script starts by creating an API object (**`wcapi`**) using the provided URL, consumer key, consumer secret, and timeout values. This object allows interaction with the WooCommerce REST API.

4. Products are downloaded from the webshop in batches of 100 products per page. The script initializes an empty dictionary (`pags_products`) to store the product data from each page. It then enters a while loop to fetch products from each page incrementally.

5. Inside the loop, the script makes a GET request to the WooCommerce API endpoint for retrieving products. The **`params`** argument is used to specify the number of products per page (**`per_page`**) and the current page number (**`page`**). The response is stored in the **`response`** variable.

6. The response is converted to JSON format using the **`json()`** method, and the resulting JSON data is stored in the **`products`** variable.

7. If there are no more products in the response (i.e., **`products`** is empty), the loop is exited.

8. The script then normalizes the product JSON data into a pandas DataFrame (`df`) using **`pd.json_normalize(products)`**.

9. The DataFrame for the current page is added to the **`pags_products`** dictionary with a key representing the page number.

10. The page number is incremented by 1 for the next iteration of the loop.

11. After all pages have been processed, the script concatenates all the DataFrames in **`pags_products`** into a single DataFrame (**`web_all_df`**) using **`pd.concat`**.

12. From the combined DataFrame, only specific columns (**`id`**, **`sku`, **`name`, **`stock_quantity`) are selected and stored in the **`web_df`** DataFrame.

13. The script prints the downloaded inventory data (**`web_df`**) and a line of dashes for visual separation.

14. Duplicate products based on the **`id`** column are searched for in **`web_df`**, and any duplicates found are removed using the **`drop_duplicates`** method.

15. The script prints whether duplicates are present in the cleaned **`web_df`** DataFrame.

16. Finally, the function returns the imported inventory as a pandas DataFrame.


In [43]:
@log_exception
def import_inventary_web(url=url,
                         consumer_key=consumer_key,
                         consumer_secret=consumer_secret,
                         timeout=timeout):
    """
    Imports the inventory from a WooCommerce webshop using the WooCommerce REST API.

    Args:
        url (str): The URL of the WooCommerce webshop.
        consumer_key (str): The consumer key for the REST API authentication.
        consumer_secret (str): The consumer secret for the REST API authentication.
        timeout (int): The request timeout in seconds.

    Returns:
        pd.DataFrame: The imported inventory as a pandas DataFrame.

    Raises:
        None

    Examples:
        >>> import_inventory_web('https://yoursiteweb.com', 'ck_****',
        >>>     'cs_****', 500)
        # The function will create a WooCommerce API object, download products from the webshop,
        # transform them into a DataFrame, remove duplicates based on product ID, and return the
        # imported inventory as a DataFrame.

    """
    print("We start with the function import_inventory_web")
    # Create the API object
    wcapi = API(url=url,
                consumer_key=consumer_key,
                consumer_secret=consumer_secret,
                timeout=timeout)
    print("Created wcapi")

    # Download products from the webshop
    pags_products = {}
    page = 1
    while True:
        print(f"Let's go for page {page} in the while loop")
        response = wcapi.get('products', params={'per_page': 100, 'page': page})
        print("Created response")
        products = response.json()
        print("Created products")
        if not products:  # No more products
            break
        df = pd.json_normalize(products)
        pags_products["page_" + str(page)] = df
        page += 1

    # Combine dataframes from the dictionary
    web_all_df = pd.concat(pags_products.values(), ignore_index=True)
    # Select the fields of interest
    web_df = web_all_df[['id', 'sku', 'name', 'stock_quantity']]
    print("Let's take a look at the data downloaded from the website\n", web_df)
    print("-"*100)

    # Search for and remove duplicates based on product ID
    web_df.drop_duplicates(subset=['id'], inplace=True)
    print("We have duplicates on the web after deleting them?\n",
            web_df[web_df.duplicated(subset=['id'])].sort_values("id"))
    return web_df


### web_edisoft_join()

The function **web_edisoft_join`** is designed to compare the stock data between two dataframes: **`web_df`** representing the stock data from a web page and **`edisoft_df`** representing the stock data from a management program called Edisoft. The function performs the following steps:

1. Joins the two dataframes based on the columns 'sku' in **`web_df`** and 'isbn' in **`edisoft_df`**. The resulting dataframe is stored in **`web_edisoft`**.

2. Prints the merged data (**`web_edisoft`**) to display the joined information.

3. Checks for missing values (**`NaN`**) in the merged dataframe and prints the count of NaN values for each column.

4. Fills the missing stock values (**`NaN`**) in the 'stock' column of **`web_edisoft`** with the corresponding values from the 'stock_quantity' column.

5. Adds a new column called 'to_save' to mark the books that have different stock quantities in both dataframes. If the 'stock_quantity' is different from the 'stock', the value in 'to_save' is set to 1; otherwise, it is set to 0.

6. Filters out the books with different stock quantities by selecting rows where the 'to_save' column is equal to 1.

7. Keeps only the 'id' and 'stock' columns in the resulting dataframe (**`web_edisoft`**) and renames the 'stock' column to 'stock_quantity'.

8. Converts the 'stock_quantity' column to the 'int32' data type.

9. Converts all values in the dataframe to strings using the **`astype('str')`** method.

10. Prints the books that need to be updated based on the differences in stock quantities, along with the number of books to be updated.

11. If no books require an update, a message is printed, and the current date and a corresponding message are written to a CSV file called "log_update_stock.csv". An email is also sent, assuming the necessary function **`enviar_email`** is defined elsewhere. The program then exits.

12. The function prints the information of the resulting dataframe (**`web_edisoft`**) using the **`info()`** method.

13. Finally, the function returns the resulting dataframe.

The function aims to identify and select the items that have different stock quantities between the web page and Edisoft, providing a filtered dataframe with the items that require updating.

In [44]:
@log_exception
def web_edisoft_join(web_df, edisoft_df):
    """
    Selects the items with different stock between two dataframes: one containing stock data from a web page and the
    other from a management program called Edisoft.

    Args:
        web_df (pd.DataFrame): DataFrame with stock data from the web page.
        edisoft_df (pd.DataFrame): DataFrame with stock data from Edisoft.

    Returns:
        pd.DataFrame: DataFrame with the items that have different stock in both dataframes.

    Raises:
        None

    Examples:
        >>> web_edisoft_join(web_df, edisoft_df)
        # The function will join the two dataframes based on the 'sku' and 'isbn' columns,
        # identify the items with different stock, and return a filtered dataframe.

    """
    # Join the datasets using 'sku' and 'isbn' as keys
    web_edisoft = web_df.set_index('sku').join(edisoft_df.set_index('isbn'), lsuffix='_web', rsuffix='_edisoft')

    print("Let's see the merged data:\n", web_edisoft)
    print("-" * 100)

    # Check for missing values after merging
    print(f"We have {web_edisoft.isna().sum().sum()} NaN values after merging\n")
    print(web_edisoft.isna().sum())

    # Fill missing stock values with the web stock_quantity
    web_edisoft['stock'] = web_edisoft['stock'].fillna(web_edisoft['stock_quantity'])
    print(web_edisoft['stock_quantity'].isna().sum())

    # Mark the books that have different stock in both dataframes
    web_edisoft['to_save'] = np.where(web_edisoft['stock_quantity'] != web_edisoft['stock'], 1, 0)

    # Filter out the books with different stock
    web_edisoft = web_edisoft[web_edisoft['to_save'] == 1]

    # Keep only the 'id' and 'stock' columns
    web_edisoft = web_edisoft[['id', 'stock']]
    web_edisoft.rename(columns={'stock': 'stock_quantity'}, inplace=True)

    # Convert stock_quantity to int
    web_edisoft['stock_quantity'] = web_edisoft['stock_quantity'].astype('int32')

    # Convert all values to strings
    web_edisoft = web_edisoft.astype('str')

    print("Let's take a look at the books to be updated:\n", web_edisoft)
    print(f"The number of books to be updated is: {web_edisoft.shape[0]}")

    if web_edisoft.shape[0] == 0:
        print("No update when Edisoft inventory matches web inventory")
        with open("log_update_stock.csv", "a", encoding="utf-8", newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow([date.today(), "No update when Edisoft inventory matches web inventory"])
            enviar_email("No update when Edisoft inventory matches web inventory")
        exit()

    print(web_edisoft.info())
    return web_edisoft


### stock_update_batch()

The function **`stock_update_batch`** is responsible for updating the stock of a web store using the WooCommerce REST API and verifying if the update has been performed correctly. Here's a breakdown of what the function does:

1. It takes several parameters: **`df`** (a Pandas DataFrame containing the stock data to be updated), **`url`** (the URL of the WooCommerce web store), **`consumer_key`** and **`consumer_secret`** (authentication credentials for the REST API), and **`timeout`** (the request timeout in seconds).

2. An API object (**`wcapi`**) is created using the provided URL, consumer key, consumer secret, and timeout.

3. The function converts the DataFrame (**`df`**) into a dictionary (**`df_dict`**). It then randomly selects a sample of items from the dictionary for checking the update.

4. The DataFrame is split into batches to avoid exceeding the connection time. Each batch is used to create a dictionary (**`stock_to_update`**) that is passed to the WooCommerce API's **`put`** method, which updates the stock of the web store.

5. After the update, the function checks if the updated items have been properly updated. It retrieves the product information from the API using the product IDs from the randomly selected items. The retrieved information is converted into a DataFrame (**`product_df`**), and the stock quantity is compared with the expected stock quantity from **`df_dict`**. If a mismatch is found, an error message is printed, and the item is recorded in a log file.

6. Finally, the function writes a summary of the update to the log file and sends an email notification.

Please note that there are a few missing parts in the code, such as the definition of the **`API`** class and the **`enviar_email`** function, which are referenced but not provided in the code. You will need to ensure that these parts are correctly defined and imported for the script to work as intended.

In [45]:
@log_exception
def stock_update_batch(df,
                       url=url,
                       consumer_key=consumer_key,
                       consumer_secret=consumer_secret,
                       timeout=timeout):
    """
    Updates the stock of a web store using the WooCommerce REST API and verifies the update.

    Args:
        df (pd.DataFrame): DataFrame containing the stock data to be updated.
        url (str): The URL of the WooCommerce web store.
        consumer_key (str): The consumer key for the REST API authentication.
        consumer_secret (str): The consumer secret for the REST API authentication.
        timeout (int): The request timeout in seconds.

    Returns:
        None

    Raises:
        None

    Examples:
        >>> stock_update_batch(df, url, consumer_key, consumer_secret, timeout)
        # The function will update the stock of the web store using the provided DataFrame,
        # and then it will check if the update has been performed correctly.

    """
    # Create the API object
    wcapi = API(url=url,
                consumer_key=consumer_key,
                consumer_secret=consumer_secret,
                timeout=timeout)

    # Create a dictionary from the DataFrame and select a random sample for checking
    df_dict = df.to_dict('records')
    number_items_to_check = math.ceil(len(df_dict) * 5 / 100)
    items_to_check = sample(range(len(df_dict)), number_items_to_check)

    # Divide the DataFrame into batches to avoid exceeding the connection time
    interval_for_cut = math.ceil(df.shape[0] / 10)
    for df_to_update in np.array_split(df, interval_for_cut):
        # Create a dictionary to pass to wcapi for updating the stock
        stock_to_update = {'update': df_to_update.to_dict('records')}
        wcapi.put("products/batch", stock_to_update).json()

    # Check if the updated items have been properly updated
    id_list = [df_dict[i]['id'] for i in items_to_check]
    stock_list = [df_dict[i]['stock_quantity'] for i in items_to_check]
    id_stock = zip(id_list, stock_list)

    for item, stock_item in id_stock:
        product = wcapi.get("products/" + str(item)).json()
        product_df = pd.json_normalize(product)
        product_df = product_df.loc[:, ('id', 'sku', 'name', 'stock_quantity')]
        stock_prod = product_df['stock_quantity'].item()

        if stock_prod != int(stock_item):
            print(f"At least the following item has not been updated: {item}")
            with open("log_update_stock.csv", "a", encoding="utf-8", newline='') as csv_file:
                csv_writer = csv.writer(csv_file)
                csv_writer.writerow([date.today(), f"At least the following item has not been updated: {item}"])
                enviar_email(f"At least the following item has not been updated: {item}")
            exit()

    with open("log_update_stock.csv", "a", encoding="utf-8", newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow([date.today(), f"The number of books that have been updated is: {df.shape[0]}"])
        enviar_email(f"The number of books that have been updated is: {df.shape[0]}")


In [46]:
edisoft_df = import_clean_inventory(path)
web_df = import_inventary_web()
web_edisoft = web_edisoft_join(web_df, edisoft_df)
stock_update_batch(df = web_edisoft)

Today's file name is W:/soft/exporsinli/I203996 and yesterday's file name            is W:/soft/exporsinli/I203953.
Let's take a look:
       D978-84-16941-15-5000502 000100962
0     D978-84-16941-16-2000520 000100962
1     D978-84-16941-17-9000681 000100962
2     D978-84-16941-18-6000198 000120192
3     D978-84-16941-19-3000452 000134615
4     D978-84-16941-20-9000536 000134615
...                                  ...
1525  D978-84-7869-608-6000048 000143750
1526  D978-84-7869-611-6000656 000095673
1527  D978-84-7869-011-4000012 000187500
1528  D978-84-7869-544-7000551 000120192
1529  D978-84-7869-612-3000313 000120192

[1530 rows x 1 columns]
----------------------------------------------------------------------------------------------------
Let's take another look:
                isbn  stock   price
0     9788416941162    520  10.096
1     9788416941179    681  10.096
2     9788416941186    198  12.019
3     9788416941193    452  13.461
4     9788416941209    536  13.461
...       

C:\Users\LUIS\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
send: 'ehlo [192.168.1.34]\r\n'
reply: b'250-plum.premium.hostns.io Hello 136.red-81-43-69.staticip.rima-tde.net [81.43.69.136]\r\n'
reply: b'250-SIZE 52428800\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-PIPECONNECT\r\n'
reply: b'250-AUTH PLAIN LOGIN\r\n'
reply: b'250 HELP\r\n'
reply: retcode (250); Msg: b'plum.premium.hostns.io Hello 136.red-81-43-69.staticip.rima-tde.net [81.43.69.136]\nSIZE 52428800\n8BITMIME\nPIPELINING\nPIPECONNECT\nAUTH PLAIN LOGIN\nHELP'
send: 'AUTH PLAIN AGx1aXNiYWxsZXN0ZXJvc0BlZGl0b3JpYWxjZXBlLmVzAFNpc2lmbyoyNzAx\r\n'
reply: b'235 Authentication succeeded\r\n'
reply: retco